In [ ]:
from __future__ import annotations

#TARGET_MODEL = "mistralai/Mistral-7B-v0.1"
#TARGET_MODEL = "HuggingFaceH4/zephyr-7b-alpha"
TARGET_MODEL = "microsoft/deberta-v3-large"

DEBUG = False

In [ ]:
# %% Directory settings

# ====================================================
# Directory settings
# ====================================================
from pathlib import Path

OUTPUT_DIR = Path("./")
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

INPUT_DIR = Path("../input/")

In [ ]:
import pandas as pd

#Import train

#train = 
#print(train.label.value_counts())

In [ ]:
print(train.shape)
display(train.head())

In [ ]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
X = train.loc[:, train.columns != "label"]
y = train.loc[:, train.columns == "label"]

for i, (train_index, valid_index) in enumerate(skf.split(X, y)):
    train.loc[valid_index, "fold"] = i
    
print(train.groupby("fold")["label"].value_counts())
train.head()

In [ ]:
train.drop(columns=["source", "prompt_name"],inplace=True)

print(train.shape)
display(train.head())

In [ ]:
train["fold"] = train["fold"].astype(int)
print(train.fold.value_counts())

In [ ]:
# fold0 as valid
val = train[train["fold"] == 0]
train = train[train["fold"] != 0]

In [ ]:
display(train.head(3))
display(val.head(3))

In [ ]:
train_df = train[["text", "label"]]
valid_df = val[["text", "label"]]
print(train_df.shape)
print(train_df.label.value_counts())
print(valid_df.shape)
print(valid_df.label.value_counts())

In [ ]:
#https://www.kaggle.com/code/hotchpotch/llm-detect-pip

!pip install -q -U peft --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U accelerate --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U transformers --no-index --find-links ../input/llm-detect-pip/

In [ ]:
# load model with 4bit bnb

from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType # type: ignore
from transformers import BitsAndBytesConfig
import torch

peft_config = LoraConfig(
    r=64,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    target_modules=[
        "q_proj",
        "v_proj"
    ],
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

#bnb_config = BitsAndBytesConfig(
#    load_in_8bit=True,
#    bnb_8bit_quant_type="nf8",
#    bnb_8bit_use_double_quant=True,
#    bnb_8bit_compute_dtype=torch.float16
#)

In [ ]:
from transformers import AutoTokenizer, LlamaForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(TARGET_MODEL, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
base_model = LlamaForSequenceClassification.from_pretrained(
    TARGET_MODEL,
    num_labels=2,
    quantization_config=bnb_config,   #TO QUANTIZE
    device_map={"":0}
)
base_model.config.pretraining_tp = 1 # 1 is 7b
base_model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
model = get_peft_model(base_model, peft_config)

In [ ]:
model.print_trainable_parameters()

In [ ]:
print(train_df.shape)
print(valid_df.shape)

In [ ]:
print(train_df.label.value_counts(), valid_df.label.value_counts())

In [ ]:
# datasets
from datasets import Dataset

# from pandas
train_ds = Dataset.from_pandas(train_df)
valid_ds = Dataset.from_pandas(valid_df)

In [ ]:
def preprocess_function(examples, max_length=512):   
    return tokenizer(examples["text"], truncation=True, max_length=max_length, padding=True)

In [ ]:
train_tokenized_ds = train_ds.map(preprocess_function, batched=True)
valid_tokenized_ds = valid_ds.map(preprocess_function, batched=True)

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest")

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    accuracy_val = accuracy_score(labels, predictions)
    roc_auc_val = roc_auc_score(labels, predictions)
    
    return {
        "accuracy": accuracy_val,
        "roc_auc": roc_auc_val,
    }

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback

steps = 5 if DEBUG else 20

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    learning_rate=5e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    max_grad_norm=0.3,
    optim='paged_adamw_32bit',
    lr_scheduler_type="cosine",
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    push_to_hub=False,
    warmup_steps=steps,
    eval_steps=steps,
    logging_steps=steps,
    report_to='none'
)

early_stopping = EarlyStoppingCallback(early_stopping_patience=2)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_ds,
    eval_dataset=valid_tokenized_ds,
    tokenizer=tokenizer,
    callbacks=[early_stopping],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
from shutil import rmtree

trainer.save_model(output_dir=str(OUTPUT_DIR))

for path in Path(training_args.output_dir).glob("checkpoint-*"):
    if path.is_dir():
        rmtree(path)

In [ ]:
del trainer, model, base_model

In [ ]:
# cuda cache clear
import torch
torch.cuda.empty_cache()

In [ ]:
# load model / tokenizer with 4bit bnb

from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType # type: ignore
from transformers import BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
from transformers import AutoTokenizer, LlamaForSequenceClassification

base_model = LlamaForSequenceClassification.from_pretrained(
    TARGET_MODEL,
    num_labels=2,
    quantization_config=bnb_config,
    device_map={"":0}
)
base_model.config.pretraining_tp = 1 # 1 is 7b
base_model.config.pad_token_id = tokenizer.pad_token_id


In [ ]:
model = PeftModel.from_pretrained(base_model, str(OUTPUT_DIR))


In [ ]:
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
pred_output = trainer.predict(valid_tokenized_ds)
logits = pred_output.predictions

In [ ]:
logits = pred_output.predictions

In [ ]:
valid_df.label

In [ ]:
logits

In [ ]:
# from scipy.special import expit as sigmoid
import numpy as np
def sigmoid(x):
    return 1 / (1 + np.exp(-x))  
probs = sigmoid(logits[:, 1])
probs.shape, probs[0:5]

In [ ]:
sub = pd.DataFrame()
sub['id'] = valid_df['id']
sub['generated'] = probs
sub.to_csv('submission.csv', index=False)
sub.head()

In [ ]:
!ls -alh /kaggle/working/